## Overview


The goal of this sprint is to develop _rich_ (HTML/CSS/JS) representations of popular objects in the 
scverse ecosystem.

We will start with [`AnnData`](https://anndata.readthedocs.io/en/latest/), the core 
object for storing single-cell data that powers [Scanpy](https://scanpy.readthedocs.io/en/stable/) - a toolkit for analyzing single-cell gene
expression.

`AnnData` is a systematic way of storing and retrieving intermediate analysis results, like 
principal components scores, UMAP embeddings, cluster labels, etc for single cell experiments.

From the [paper](https://www.biorxiv.org/content/10.1101/2021.12.16.473007v1.full.pdf),

> The anndata library provides a canonical data structure for book-keeping [both original and learned single-cell annotations, as well as task-associated representations], a capability not addressed by pandas (McKinney, 2010), xarray (Hoyer & Hamman, 2017), or commonly-used modeling packages like scikit-learn (Pedregosa et al., 2011).


TL;DR - `AnnData` is a special container around several popular libraries (pandas, dask, xarray, scikit-learn) 
that provides a standard way to operate on and store single-cell data.

## AnnData Model

Annotated data are stored as follows:

<img width="300" src="https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg" />

- `X`: Main data matrix with observations as rows and variables as columns (Array)
- `obs`: One-dimensional annotations for observations (DataFrame)
- `var`: One-dimensional annotations for variables (DataFrame)
- `obsm`: Multi-dimensional annotations for observations (Dictionary)
- `varm`: Multi-dimensional annotations for variables (Dictionary)
- `obsp`: Pairwise relationships among observations (Sparse Matrix)
- `varp`: Pairwise relationships among variables (Sparse Matrix)
- `uns`: Unstructured data associated with the dataset (Dictionary)



## Motivation

As you can see, `AnnData` objects can be complex, with many related data structures and features.

Let's take a closer look at an `AnnData` object, specifically this PBMC 3K dataset from the 
AnnData tutorial.

In [1]:
import anndata
import pooch

# download a dataset
datapath = pooch.retrieve(
    url="https://figshare.com/ndownloader/files/40067737",
    known_hash="md5:b80deb0997f96b45d06f19c694e46243",
    path="../data",
    fname="scverse-getting-started-anndata-pbmc3k_processed.h5ad",
)
adata = anndata.read_h5ad("../data/scverse-getting-started-anndata-pbmc3k_processed.h5ad")
adata

AnnData object with n_obs × n_vars = 2638 × 11505
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain_cell_types'
    var: 'gene_names', 'n_cells', 'gene_ids'
    uns: 'louvain', 'louvain_colors', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'raw'
    obsp: 'distances_all'

As you can see, we have a nice high-level overview of the dataset with this simple text `__repr__`,
but visually inspecting various components like the main data matrix (`X`), annotations (`obs`, `vars`),
and related structures requires executing additional cells to get the nested _representations_ of the
underlying data objects.



In [2]:
adata.X

<2638x11505 sparse matrix of type '<class 'numpy.float32'>'
	with 2076576 stored elements in Compressed Sparse Row format>

In [3]:
adata.obs.head()

,n_genes,percent_mito,n_counts,louvain_cell_types
cell_barcode,,,,
AAACATACAACCAC-1,781,0.030178,2419.0,CD4 T cells
AAACATTGAGCTAC-1,1352,0.037936,4903.0,B cells
AAACATTGATCAGC-1,1131,0.008897,3147.0,CD4 T cells
AAACCGTGCTTCCG-1,960,0.017431,2639.0,CD14+ Monocytes
AAACCGTGTATGCG-1,522,0.012245,980.0,NK cells


In [4]:
adata.var.head()

,gene_names,n_cells,gene_ids
gene_names,,,
LINC00115,LINC00115,18,ENSG00000225880
NOC2L,NOC2L,258,ENSG00000188976
KLHL17,KLHL17,9,ENSG00000187961
PLEKHN1,PLEKHN1,7,ENSG00000187583
HES4,HES4,145,ENSG00000188290


### What can we do?

IPython (and thus Jupyter) has [special "hooks"](https://ipython.readthedocs.io/en/stable/config/integrating.html) to display richer representations of objects using technologies beyond simple text. This feature let's you integrate your Python objects.

In fact, you are seeing this feature with the Pandas dataframe used for `X.obs` above. See how when we just run the cell:


In [5]:
adata.obs

,n_genes,percent_mito,n_counts,louvain_cell_types
cell_barcode,,,,
AAACATACAACCAC-1,781,0.030178,2419.0,CD4 T cells
AAACATTGAGCTAC-1,1352,0.037936,4903.0,B cells
AAACATTGATCAGC-1,1131,0.008897,3147.0,CD4 T cells
AAACCGTGCTTCCG-1,960,0.017431,2639.0,CD14+ Monocytes
AAACCGTGTATGCG-1,522,0.012245,980.0,NK cells
...,...,...,...,...
TTTCGAACTCTCAT-1,1155,0.021104,3459.0,CD14+ Monocytes
TTTCTACTGAGGCA-1,1227,0.009294,3443.0,B cells
TTTCTACTTCCTCG-1,622,0.021971,1684.0,B cells


We get a richly formatted table in the notebook that uses HTML/CSS. We can hover rows
and get high-level information about our data. This is possible because the `pd.DataFrame` object implments
some "special" methods to alert Jupyter it has a _rich_, HTML-based representation of the object.

Jupyter detects these methods and then will prefer to render this nicer representation within a web-based notebook. 
Other environments may need to render the raw text (if they run outside of the browser). To access the original
text representation, we can still call `print()` or `repr()` on the object:

In [6]:
print(adata.obs)

                  n_genes  percent_mito  n_counts louvain_cell_types
cell_barcode                                                        
AAACATACAACCAC-1      781      0.030178    2419.0        CD4 T cells
AAACATTGAGCTAC-1     1352      0.037936    4903.0            B cells
AAACATTGATCAGC-1     1131      0.008897    3147.0        CD4 T cells
AAACCGTGCTTCCG-1      960      0.017431    2639.0    CD14+ Monocytes
AAACCGTGTATGCG-1      522      0.012245     980.0           NK cells
...                   ...           ...       ...                ...
TTTCGAACTCTCAT-1     1155      0.021104    3459.0    CD14+ Monocytes
TTTCTACTGAGGCA-1     1227      0.009294    3443.0            B cells
TTTCTACTTCCTCG-1      622      0.021971    1684.0            B cells
TTTGCATGAGAGGC-1      454      0.020548    1022.0            B cells
TTTGCATGCCTCAC-1      724      0.008065    1984.0        CD4 T cells

[2638 rows x 4 columns]


IPython supports enhanced object representations through custom `_repr_*_()` methods, enabling objects to be displayed in formats beyond standard text. If these methods are absent or return `None`, the basic (text) `repr()` is used.

Multiple formats can be defined, and the UI chooses which to display, without side effects.
Each method should return data in its specific format.


| Format             | REPL | Notebook | Qt Console |
|--------------------|------|----------|------------|
| `_repr_pretty_`    | yes  | yes      | yes        |
| `_repr_svg_`       | no   | yes      | yes        |
| `_repr_png_`       | no   | yes      | yes        |
| `_repr_jpeg_`      | no   | yes      | yes        |
| `_repr_html_`      | no   | yes      | no         |
| `_repr_javascript_`| no   | yes      | no         |
| `_repr_markdown_`  | no   | yes      | no         |
| `_repr_latex_`     | no   | yes      | no         |
| `_repr_mimebundle_`| no   | ?        | ?          |

We can see that the `adata.obs` implements the `_repr_html_` method, which returns HTML. Jupyter detects this method and automatically calls it in the call. We can see this final string, just by calling it ourselves:


In [7]:
print(adata.obs.head(2)._repr_html_())

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>n_genes</th>
      <th>percent_mito</th>
      <th>n_counts</th>
      <th>louvain_cell_types</th>
    </tr>
    <tr>
      <th>cell_barcode</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>AAACATACAACCAC-1</th>
      <td>781</td>
      <td>0.030178</td>
      <td>2419.0</td>
      <td>CD4 T cells</td>
    </tr>
    <tr>
      <th>AAACATTGAGCTAC-1</th>
      <td>1352</td>
      <td>0.037936</td>
      <td>4903.0</td>
      <td>B cells</td>
    </tr>
  </tbody>
</table>
</div>


## Your first `_html_repr_`

We can implement a custom repr on a simple object of our own:

In [8]:
class Welcome:
    def __init__(self, name):
        self.name = name 

    def _repr_html_(self):
        return f"""
        <style>
            .scverse-hackathon {{
                max-width: 300px;
                display: grid;
                place-items: center;
            }}
        </style>
        <div class="scverse-hackathon">
            <img src="https://hms-dbmi.github.io/scverse-hackathon-spring-2024/logo.svg" />
            <p>Welcome to the hackathon, {self.name}!</p>
        </div>
        """

Welcome(name="Lisa Simpson")

## Other libraries / inspiration

Both dask and xarray provide very nice, expandable HTML reprs for the objects associated with these frameworks. I think these could serve as a source of inspriation for our work with `AnnData`.

For example, Dask provides a high-level view of mulidimenional arrays with some additional metadata about memory/chunk sizes:

In [9]:
import numpy as np
import xarray as xr
import dask.array as da

data = np.random.randn(2, 10_000)
ddata = da.array(data)
ddata # dask array

dask.array<array, shape=(2, 10000), dtype=float64, chunksize=(2, 10000), chunktype=numpy.ndarray>

The `xarray.DataArray` extends this further by adding _collapsable_ metadata about the different additional components beyond the main mulidimensional array container. I think xarray's notion of coordinates could be a real inspriation for our work.

In [10]:
xrdata = xr.DataArray(ddata, dims=("x", "y"), coords={"x": [10, 20]})
xrdata

<xarray.DataArray 'array-8ecb1f6e7e10f1747ede93a81db51959' (x: 2, y: 10000)> Size: 160kB
dask.array<array, shape=(2, 10000), dtype=float64, chunksize=(2, 10000), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) int64 16B 10 20
Dimensions without coordinates: y

Click on the arrows and the database icons to expand and collapse fields.

This is very nice user experience to try to understand the dataset at a high-level. Another benefit is that SVG-based graphics can be exported and embedded as figures to display the datasets!

You can see a more complete version of a `xarray.DataArray` with their built-in example dataset:


In [11]:
ds = xr.tutorial.load_dataset("air_temperature")
ds.air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)> Size: 31MB
array([[[241.2 , 242.5 , 243.5 , ..., 232.8 , 235.5 , 238.6 ],
        [243.8 , 244.5 , 244.7 , ..., 232.8 , 235.3 , 239.3 ],
        [250.  , 249.8 , 248.89, ..., 233.2 , 236.39, 241.7 ],
        ...,
        [296.6 , 296.2 , 296.4 , ..., 295.4 , 295.1 , 294.7 ],
        [295.9 , 296.2 , 296.79, ..., 295.9 , 295.9 , 295.2 ],
        [296.29, 296.79, 297.1 , ..., 296.9 , 296.79, 296.6 ]],

       [[242.1 , 242.7 , 243.1 , ..., 232.  , 233.6 , 235.8 ],
        [243.6 , 244.1 , 244.2 , ..., 231.  , 232.5 , 235.7 ],
        [253.2 , 252.89, 252.1 , ..., 230.8 , 233.39, 238.5 ],
        ...,
        [296.4 , 295.9 , 296.2 , ..., 295.4 , 295.1 , 294.79],
        [296.2 , 296.7 , 296.79, ..., 295.6 , 295.5 , 295.1 ],
        [296.29, 297.2 , 297.4 , ..., 296.4 , 296.4 , 296.6 ]],

       [[242.3 , 242.2 , 242.3 , ..., 234.3 , 236.1 , 238.7 ],
        [244.6 , 244.39, 244.  , ..., 230.3 , 232.  , 235.7 ],
        [256.2 , 255.5 , 254.2 , ..., 231.2 , 233.2 , 238.2 ],
        ...,
...
        ...,
        [294.79, 295.29, 297.49, ..., 295.49, 295.39, 294.69],
        [296.79, 297.89, 298.29, ..., 295.49, 295.49, 294.79],
        [298.19, 299.19, 298.79, ..., 296.09, 295.79, 295.79]],

       [[245.79, 244.79, 243.49, ..., 243.29, 243.99, 244.79],
        [249.89, 249.29, 248.49, ..., 241.29, 242.49, 244.29],
        [262.39, 261.79, 261.29, ..., 240.49, 243.09, 246.89],
        ...,
        [293.69, 293.89, 295.39, ..., 295.09, 294.69, 294.29],
        [296.29, 297.19, 297.59, ..., 295.29, 295.09, 294.39],
        [297.79, 298.39, 298.49, ..., 295.69, 295.49, 295.19]],

       [[245.09, 244.29, 243.29, ..., 241.69, 241.49, 241.79],
        [249.89, 249.29, 248.39, ..., 239.59, 240.29, 241.69],
        [262.99, 262.19, 261.39, ..., 239.89, 242.59, 246.29],
        ...,
        [293.79, 293.69, 295.09, ..., 295.29, 295.09, 294.69],
        [296.09, 296.89, 297.19, ..., 295.69, 295.69, 295.19],
        [297.69, 298.09, 298.09, ..., 296.49, 296.19, 295.69]]])
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

Notice how we could expand/collapse obs & var for `AnnData` in a table like _Coordinates_.